# Mask R-CNN - Train on Shapes Dataset


This notebook shows how to train Mask R-CNN on your own dataset. To keep things simple we use a synthetic dataset of shapes (squares, triangles, and circles) which enables fast training. You'd still need a GPU, though, because the network backbone is a Resnet101, which would be too slow to train on a CPU. On a GPU, you can start to get okay-ish results in a few minutes, and good results in less than an hour.

The code of the *Shapes* dataset is included below. It generates images on the fly, so it doesn't require downloading any data. And it can generate images of any size, so we pick a small image size to train faster. 

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

from skimage.draw import rectangle

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

DATA_DIR = os.path.join(ROOT_DIR, "../data")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

#Make GPUs visible
# !export HIP_VISIBLE_DEVICES=0,1,2,3

# Download COCO trained weights from Releases if needed
# if not os.path.exists(COCO_MODEL_PATH):
#     utils.download_trained_weights(COCO_MODEL_PATH)

Using TensorFlow backend.


In [ ]:
import mlflow
#add mlflow stuff

MLFLOW_SERVER = 'occ01ap200.na.simplot.com'

os.environ['NO_PROXY'] = MLFLOW_SERVER
mlflow.tracking.set_tracking_uri('http://' + MLFLOW_SERVER + ':5005')
EXPERIMENT_NAME = 'kaggle_openimage_mask_rcnn_v1.0'
mlflow.set_experiment(EXPERIMENT_NAME)

# os.environ['AZURE_STORAGE_ACCESS_KEY'] = ''

## Pull in annotations from database (from Simplot)

In [2]:
import pyodbc
import os
import pandas as pd

DATA_DIR = os.path.join(ROOT_DIR, "../data")

COCO_MODEL_PATH = os.path.join(DATA_DIR, "mask_rcnn_coco.h5")

ANNOTATION_COLS = ['XMax','XMin','YMin','YMax','LabelDescription']

passwd = os.environ['dgs_sandbox_pwd']

conn = pyodbc.connect('DSN=BIdevDatabase;'
                      'Database=Sandbox;'
                      'UID=DGSuser;'
                      'PWD=' + passwd)


class_descriptions = pd.read_sql("SELECT LabelName, LabelDescription from [kaggle].[Class_Description]",conn)

#add 1 since Background class is automatically added at index 0
class_descriptions['LabelID'] = class_descriptions.index + 1

display(class_descriptions.head())

#This now holds the annotation information.
bboxes = pd.read_sql("SELECT ImageID, XMax, XMin, YMin, YMax, LabelName FROM [Sandbox].[kaggle].[Combined_Set_Detection_BBox]", conn)

annotations = pd.merge(bboxes,class_descriptions, on='LabelName',how='inner')

display(annotations.head())

# This now holds the list of images
image_paths = pd.read_sql("SELECT ImageID, RelativePath, Height, Width, Mode from [kaggle].[Image_Path]", conn)

display(image_paths.head())

# Inner join on the two dataframes, so we now have images combined with associated annotations
annotated_image_paths = pd.merge(image_paths,annotations, on='ImageID',how='inner')

display(annotated_image_paths.head())

,LabelName,LabelDescription,LabelID
0,/m/061hd_,Infant bed,1
1,/m/06m11,Rose,2
2,/m/03120,Flag,3
3,/m/01kb5b,Flashlight,4
4,/m/0120dh,Sea turtle,5


,ImageID,XMax,XMin,YMin,YMax,LabelName,LabelDescription,LabelID
0,5a27bbe9ab1ede93,0.706875,0.026250,0.553571,0.999060,/m/04p0qw,Billiard table,47
1,2fe71cd9dc2acfdd,0.999333,0.000000,0.000000,0.998565,/m/04p0qw,Billiard table,47
2,ea2422b5f4c97b42,0.868228,0.000000,0.168945,0.431641,/m/04p0qw,Billiard table,47
3,ea2422b5f4c97b42,0.888726,0.000000,0.567383,0.999023,/m/04p0qw,Billiard table,47
4,2fef02bb3916fec3,0.481250,0.045833,0.402778,0.687963,/m/04p0qw,Billiard table,47


,ImageID,RelativePath,Height,Width,Mode
0,95259d687d440013,train/95259d687d440013.jpg,1024,1024,RGB
1,ccce42a1f4ed2c96,train/ccce42a1f4ed2c96.jpg,1024,768,RGB
2,30723bdea123ccdf,validation/30723bdea123ccdf.jpg,683,1024,RGB
3,cba0bd46fcddd85d,train/cba0bd46fcddd85d.jpg,1024,1024,RGB
4,cc88539b5c74a0c0,train/cc88539b5c74a0c0.jpg,1024,1024,RGB


,ImageID,RelativePath,Height,Width,Mode,XMax,XMin,YMin,YMax,LabelName,LabelDescription,LabelID
0,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.697656,0.089844,0.000000,0.936458,/m/04yx4,Man,114
1,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.388281,0.221875,0.000000,0.309375,/m/0dzct,Human face,280
2,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.960156,0.778906,0.000000,0.300000,/m/0dzct,Human face,280
3,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.332812,0.000000,0.864583,0.998958,/m/050gv4,Plate,473
4,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.946875,0.605469,0.836458,0.998958,/m/050gv4,Plate,473


In [3]:
annotated_image_paths.head(100)

,ImageID,RelativePath,Height,Width,Mode,XMax,XMin,YMin,YMax,LabelName,LabelDescription,LabelID
0,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.697656,0.089844,0.000000,0.936458,/m/04yx4,Man,114
1,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.388281,0.221875,0.000000,0.309375,/m/0dzct,Human face,280
2,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.960156,0.778906,0.000000,0.300000,/m/0dzct,Human face,280
3,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.332812,0.000000,0.864583,0.998958,/m/050gv4,Plate,473
4,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.946875,0.605469,0.836458,0.998958,/m/050gv4,Plate,473
5,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.999219,0.904688,0.971875,0.998958,/m/050gv4,Plate,473
6,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.332031,0.000000,0.866667,0.998958,/m/03q5c7,Saucer,248
7,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.943750,0.602344,0.832292,0.998958,/m/03q5c7,Saucer,248
8,2fef4dd2f83feb18,train/2fef4dd2f83feb18.jpg,768,1024,RGB,0.730469,0.521875,0.544792,0.807292,/m/01fh4r,Teapot,464
9,307002c02f4e6862,train/307002c02f4e6862.jpg,683,1024,RGB,0.984028,0.717361,0.020833,0.970833,/m/04yx4,Man,114


## Configurations

In [ ]:
class KaggleConfig(Config):
    
    NAME = "kaggle"

    #TODO experiment with this
    GPU_COUNT = 2
    IMAGES_PER_GPU = 2  # TODO how many can we use, authors had 2 for 12 GB, we have 32 GB so ...

    # Number of classes (including background)
    NUM_CLASSES = 1 + len(class_descriptions)  # + 1 for background

    #TODO experiment with this
    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.75
    
config = KaggleConfig()
config.display()

## Subclass utils.Dataset for entire experiment

In [ ]:
class FullKaggleImageDataset(utils.Dataset):
         
    def load_kaggle_images(self, dataset_dir, grouped_by_images):
        """Load a subset of the image dataset.
        dataset_dir: The root directory of the image dataset.
        classes: Dataframe : If provided, only loads images that have the given classes.
        """

        # Add classes, BG is automatically added at index 0
        for _,row in class_descriptions.iterrows():
            self.add_class("openImages", row['LabelID'], row['LabelDescription'])
        
#         X = 0
        
        # Add images
        for i,df in grouped_by_images:    
            row = df.iloc[0]
            
#             X += 1
#             if X%10000 == 0:
#                 print(X)
            
            self.add_image(
                "openImages", image_id=i,
                path=os.path.join(dataset_dir, row['RelativePath']),
                width=row["Width"],
                height=row["Height"],
                annotations=df)
    

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        
        # Create rectangular bounding box since we are doing object detection, not segmentation
        # desired dimension is [height, width, instance_count]
        img = self.image_info[image_id]
        
        mask = np.zeros([img["height"], img["width"], len(img["annotations"])],
                        dtype=np.uint8)
        
        for i,(_,p) in enumerate(img["annotations"].iterrows()):
            # Create rectangular bounding box since we are doing object detection, not segmentation
    
            xmax = int(img["width"]*p['XMax'])
            xmin = int(img["width"]*p['XMin'])
            ymin = int(img["height"]*p['YMin'])
            ymax = int(img["height"]*p['YMax'])
            
            start = (ymin, xmin)  #top left corner ... are coordinates reversed?
            end = (ymax, xmax)  #height and width
            rr, cc = rectangle(start, end=end, shape=(img["height"],img["width"]))
            
            mask[rr, cc, i] = 1

        
        # Return mask, and array of class IDs of each instance.
        return mask.astype(np.bool), np.array(img['annotations']['LabelID'].values, dtype=np.int32)

    def image_reference(self, image_id):
        return self.image_info[image_id]['path']
    

## Utility Functions

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load dataset and display samples

### Instantiate the Full Kaggle Image Dataset classes

In [ ]:
val_paths = annotated_image_paths[annotated_image_paths['RelativePath'].str.contains('validation',regex=False)]
train_paths = annotated_image_paths[annotated_image_paths['RelativePath'].str.contains('train',regex=False)]

val_grouped = val_paths.groupby('ImageID')
train_grouped = train_paths.groupby('ImageID')

# Training dataset
dataset_train = FullKaggleImageDataset()
dataset_train.load_kaggle_images(DATA_DIR, train_grouped)
dataset_train.prepare()

# Validation dataset
dataset_val = FullKaggleImageDataset()
dataset_val.load_kaggle_images(DATA_DIR, val_grouped)
dataset_val.prepare()

In [ ]:
# Load and display random samples ... sanity check for data load
image_ids = np.random.choice(dataset_train.image_ids, 10)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

## Create Model

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.

def log_params(pz):
    for k,v in pz.items():
        mlflow.log_param(k,v)

with mlflow.start_run():
    model.train(dataset_train, dataset_val, 
                learning_rate=config.LEARNING_RATE, 
                epochs=50, 
                layers='heads')
    
    log_params(config)
    
    
# mlflow.log_param('Code_Version_Git_Hash',git_hash)

# predictions = clf.predict(X_test)

# mlflow.log_metric('AUC of ROC', skm.roc_auc_score(y_test,predictions))

# pickle.dump(clf,open(model_filepath,'wb'))
# mlflow.log_artifact(model_filepath)

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=2, 
            layers="all")

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
# model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
# model.keras_model.save_weights(model_path)

## Detection

In [ ]:
class InferenceConfig(KaggleConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
    BATCH_SIZE = 10

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
#test on a few random images
image_ids = np.random.choice(dataset_val.image_ids, inference_config.BATCH_SIZE)

images = []

for iid in image_ids:
    original_image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset_val, inference_config, iid, use_mini_mask=False)

    visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))
    
    images.append(original_image)


In [ ]:
results = model.detect(images, verbose=0)

for i in range(len(results)):
    r = results[i]
    visualize.display_instances(images[i], r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, r['scores'])

## Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 500)

#each input is a tuple of form : image, image_meta, gt_class_id, gt_bbox, gt_mask
inputs = [modellib.load_image_gt(dataset_val, inference_config, iid, use_mini_mask=False) for iid in image_ids]

APs = []

n = inference_config.BATCH_SIZE

for i in range(0,len(image_ids),n): 

    curr_inputs = inputs[i:i+n]
    
    results = model.detect([inp[0] for inp in curr_inputs], verbose=0)
    
    for j in range(len(results)):
        r = results[j]
        # Compute AP
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(curr_inputs[j][3], curr_inputs[j][2], curr_inputs[j][4],
                             r["rois"], r["class_ids"], r["scores"], r['masks'])
        APs.append(AP)
    
print("mAP: ", np.mean(APs))